## Classifier example

In [1]:
import importlib
import numpy as np
import tmd

In [2]:
list_of_modules = ["discriminant_analysis", "discriminant_analysis", "tree"]

list_of_classifiers = [
    "LinearDiscriminantAnalysis",
    "QuadraticDiscriminantAnalysis",
    "DecisionTreeClassifier",
]

In [3]:
def train(mod, classifier, data, labels, **kwargs):
    """Trains the classifier from mod of sklearn with data and targets.

    Returns a fited classifier.
    """
    clas_mod = importlib.import_module("sklearn." + mod)
    clf = getattr(clas_mod, classifier)()
    clf.set_params(**kwargs)

    clf.fit(data, labels)

    return clf

In [4]:
def predict(clf, data):
    """Predict label for data for the trained classifier clf.

    Returns the index of the predicted class for each datapoint in data.
    """
    predict_label = clf.predict([data])

    return predict_label[0]

In [25]:
def classify_cell_in_groups(
    list_of_groups=["L2_IPC", "L2_TPC_A"],
    cell_to_classify="Data/Reconstructed/L2_TPC_A/C280998A-P3.asc",
    neurite_type="apical_dendrite",
    classifier_module=list_of_modules[0],
    classifier_method=list_of_classifiers[0],
    number_of_trials=20,
):
    """Classify the cells in groups."""
    
    # ------------------------ Training dataset --------------------------------
    # Load all data from selected folders
    groups = [tmd.io.load_population(i) for i in list_of_groups]
    # Define labels depending on the number of neurons in each folder
    labels = [i + 1 for i, k in enumerate(groups) for j in k.neurons]
    # Generate a persistence diagram per neuron
    pers_diagrams = [
        tmd.methods.get_ph_neuron(j, neurite_type=neurite_type)
        for i, k in enumerate(groups)
        for j in k.neurons
    ]
    # Define x-ylimits
    xlim, ylim = tmd.analysis.define_limits(pers_diagrams)
    # Generate a persistence image for each diagram
    pers_images = [
        tmd.analysis.persistence_image_data(p, xlim=xlim, ylim=ylim) for p in pers_diagrams
    ]
    # Create the train dataset from the flatten images
    train_dataset = [i.flatten() for i in pers_images]

    # ------------------------ Test cell dataset -------------------------------
    # Load cell to be classified
    neuron2test = tmd.io.load_neuron_from_morphio(cell_to_classify)
    # Get persistence diagram from test cell
    pers2test = tmd.methods.get_ph_neuron(neuron2test, neurite_type=neurite_type)
    # Get persistence image from test cell
    pers_image2test = tmd.analysis.persistence_image_data(pers2test, xlim=xlim, ylim=ylim)
    # Create the test dataset from the flatten image of the test cell
    test_dataset = pers_image2test.flatten()

    predict_labels = []
    # Train classifier with training images for selected number_of_trials
    for i in range(number_of_trials):
        clf = train(classifier_module, classifier_method, train_dataset, labels)
        # Test classifier with test image and return predictions
        predict_labels.append(predict(clf, test_dataset))

    percentages = {
        groups[i - 1].name: np.float(len(np.where(np.array(predict_labels) == i)[0]))
        / len(predict_labels)
        for i in np.unique(labels)
    }

    return percentages

In [26]:
classify_cell_in_groups(
    list_of_groups=["Data/Reconstructed/L2_IPC", "Data/Reconstructed/L2_TPC_A"],
    cell_to_classify="Data/Reconstructed/L2_TPC_A/C280998A-P3.asc",
    neurite_type="apical_dendrite",
    classifier_module=list_of_modules[0],
    classifier_method=list_of_classifiers[0],
    number_of_trials=20,
)

Warning: {filename} is not a valid h5, swc or asc file. If asc set use_morphio to True.